# hw 6


<div>Студент Чешев А. Д.</div>


In [2]:
// 1.

val sc = spark.sparkContext
val imdbSpark = SparkSession.builder().appName("IMDB").config(sc.getConf).getOrCreate()

val filePathTest = "datasets/IMDBSentimentAnalysis/Test.csv"
val filePathTrain = "datasets/IMDBSentimentAnalysis/Train.csv"
val filePathValid = "datasets/IMDBSentimentAnalysis/Valid.csv"

val imdbTrainDataset: DataFrame = imdbSpark.read.option("header", "true").csv(filePathTrain)
val imdbTestDataset: DataFrame = imdbSpark.read.option("header", "true").csv(filePathTest)
val imdbValidDataset: DataFrame = imdbSpark.read.option("header", "true").csv(filePathValid)

In [8]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions.col

// Метод, который оставляет только значения "0" и "1" в колонке label.
def filterLabels(df: DataFrame): DataFrame = {
  df.withColumn("label", col("label").cast("int")).filter("label == 0 OR label == 1")
}

// Получаем отфильтрованные датасеты.
val imdbTrainFiltered = filterLabels(imdbTrainDataset)
val imdbTestFiltered = filterLabels(imdbTestDataset)
val imdbValidFiltered = filterLabels(imdbValidDataset)

In [4]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, SparkSession}

// Создание экземпляра Tokenizer для разделения текста на токены.
val tokenizer = new Tokenizer().setInputCol("text").setOutputCol("words")
// Создание экземпляра HashingTF для преобразования токенов в вектор признаков с использованием хэширования.
// В данном случае эмпирическим путем выбрано наиболее оптимальное значение параметара NumFeatures = 11500.
val hashingTF = new HashingTF().setInputCol("words").setOutputCol("features").setNumFeatures(11500)
// Создание экземпляра LogisticRegression с установкой параметров обучения.
val lr = new LogisticRegression().setMaxIter(10).setRegParam(0.01)
// Создание конвейера (pipeline), включающего в себя последовательное применение Tokenizer, HashingTF и LogisticRegression.
val pipeline = new Pipeline().setStages(Array(tokenizer, hashingTF, lr))

// Обучим модель.
val model = pipeline.fit(imdbTrainFiltered)

// Протестируем на тестовой выборке.
val predictions = model.transform(imdbTestFiltered)
// Выведем полученные предсказания для визуального сравнения результатов.
predictions.select("text", "label", "prediction").show()

model.write.overwrite().save("imdb_model")

+--------------------+-----+----------+
|                text|label|prediction|
+--------------------+-----+----------+
|1st watched 12/7/...|    0|       0.0|
|I saw a screening...|    0|       1.0|
|William Hurt may ...|    1|       1.0|
|IT IS A PIECE OF ...|    0|       0.0|
|I'M BOUT IT(1997)...|    0|       0.0|
|I really enjoyed ...|    1|       0.0|
|This movie was re...|    0|       0.0|
|I felt asleep, wa...|    0|       0.0|
|Brass pictures (m...|    0|       1.0|
|My interest was r...|    1|       1.0|
|Love Jones clever...|    1|       1.0|
|This is a funny f...|    1|       1.0|
|Like several othe...|    1|       1.0|
|Film version of S...|    0|       0.0|
|Spoken like a tru...|    0|       0.0|
|I think it was an...|    0|       0.0|
|This movie is goo...|    1|       1.0|
|I vaguely remembe...|    0|       0.0|
|One of the finest...|    1|       1.0|
|We watched this i...|    1|       1.0|
+--------------------+-----+----------+
only showing top 20 rows



In [6]:
// 2.

// Загрузим ранее созданную модель.
val loadedModel = PipelineModel.load("imdb_model")

// Предсказания на valid датасете.
val predictionsOnValidDataset = loadedModel.transform(imdbValidFiltered)
predictionsOnValidDataset.select("text", "label", "prediction").show()

+--------------------+-----+----------+
|                text|label|prediction|
+--------------------+-----+----------+
|someone needed to...|    0|       0.0|
|The Guidelines st...|    0|       0.0|
|This movie is a m...|    0|       0.0|
|Before Stan Laure...|    0|       1.0|
|This is the best ...|    1|       1.0|
|Somebody mastered...|    1|       1.0|
|Why did I waste 1...|    0|       0.0|
|This film takes y...|    1|       1.0|
|The Russian space...|    0|       0.0|
|I had seen 'Kalif...|    1|       1.0|
|I really enjoyed ...|    0|       1.0|
|Hi, Everyone, Oh,...|    0|       0.0|
|It takes a while ...|    1|       1.0|
|If you're one of ...|    0|       1.0|
|This is better th...|    0|       0.0|
|The first time I ...|    0|       1.0|
|One of the great ...|    1|       1.0|
|An excellent fami...|    1|       1.0|
|The next-to-last ...|    1|       1.0|
|Today You Die was...|    0|       0.0|
+--------------------+-----+----------+
only showing top 20 rows



In [7]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

// Создание экземпляра оценщика для многоклассовой классификации.
// Устанавливаются параметры:
//  - столбец с истинными метками ("label");
//  - столбец с предсказанными метками ("prediction").
// Выбирается метрика:
//  - метрика оценки качества - точность ("accuracy").
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")

// Вычислим характеристику accuracy.
val accuracy = evaluator.evaluate(predictionsOnValidDataset)
println(s"Accuracy: ${accuracy * 100}")

Accuracy: 84.31506849315068
